In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
import time
from selenium.common.exceptions import NoSuchElementException
import pandas as pd
import re
import os

# Path ke ChromeDriver kamu
chrome_driver_path = "E:\Liga 1 Fix\chromedriver-win64\chromedriver.exe"

#Setup driver
options = webdriver.ChromeOptions()
#options.add_argument("--headless")  # Tanpa tampilan GUI
service = Service(chrome_driver_path)
driver = webdriver.Chrome(service=service, options=options)
driver.maximize_window()

xpath_dict_default = {
    "Nama": '/html/body/div[1]/main/div[2]/div/div[2]/div[1]/div[1]/div[2]/div/div[1]/div/div[1]/h2',
    "Att": '/html/body/div[1]/main/div[2]/div/div[2]/div[2]/div[1]/div/div[2]/div[1]/div/div[1]/div/div/span',
    "Tec": '/html/body/div[1]/main/div[2]/div/div[2]/div[2]/div[1]/div/div[2]/div[1]/div/div[2]/div/div/span',
    "Tac": '/html/body/div[1]/main/div[2]/div/div[2]/div[2]/div[1]/div/div[2]/div[1]/div/div[3]/div/div/span',
    "Def": '/html/body/div[1]/main/div[2]/div/div[2]/div[2]/div[1]/div/div[2]/div[1]/div/div[4]/div/div/span',
    "Cre": '/html/body/div[1]/main/div[2]/div/div[2]/div[2]/div[1]/div/div[2]/div[1]/div/div[5]/div/div/span',
    "Total Bermain": '/html/body/div[1]/main/div[2]/div/div[2]/div[2]/div[2]/div[2]/div[4]/div/div/div/div/div[1]/span[2]',
    "Starting Eleven": '/html/body/div[1]/main/div[2]/div/div[2]/div[2]/div[2]/div[2]/div[4]/div/div/div/div/div[2]/span[2]',
    "Total Menit": '/html/body/div[1]/main/div[2]/div/div[2]/div[2]/div[2]/div[2]/div[4]/div/div/div/div/div[4]/span[2]',
}

xpath_dict_alternative = {
    "Nama": '/html/body/div[1]/main/div[2]/div/div[2]/div[1]/div[1]/div[2]/div/div[1]/div/div[1]/h2',
    "Total Bermain": '/html/body/div[1]/main/div[2]/div/div[2]/div[2]/div[1]/div[2]/div[4]/div/div/div/div/div[1]/span[2]',
    "Starting Eleven": '/html/body/div[1]/main/div[2]/div/div[2]/div[2]/div[1]/div[2]/div[4]/div/div/div/div/div[2]/span[2]',
    "Total Menit": '/html/body/div[1]/main/div[2]/div/div[2]/div[2]/div[1]/div[2]/div[4]/div/div/div/div/div[4]/span[2]',
}

nggak_ada = []

folder_path = "Link Pemain"
csv_files = [file for file in os.listdir(folder_path) if file.endswith(".csv")]

for file in csv_files[:2]:
    hasil_data = []
    file_path = os.path.join(folder_path, file)
    print(file)
    data = pd.read_csv(file_path)
    link_pemain = data['Link Pemain'].tolist()
    data_pemain = []

    for link in link_pemain[:2]:
        driver.get(link)
        time.sleep(3)
        pemain_data = {"Link Pemain": link}

        try:
            # Coba cari elemen "Att"
            att_elem = driver.find_element(By.XPATH, xpath_dict_default["Att"])
            pemain_data["Att"] = att_elem.text

            # Jika ditemukan, ambil semua data dari xpath_dict_default
            for key, xpath in xpath_dict_default.items():
                if key == "Att" or key == "Rating":
                    continue
                try:
                    pemain_data[key] = driver.find_element(By.XPATH, xpath).text
                except NoSuchElementException:
                    pemain_data[key] = None

        except NoSuchElementException:
            # Jika "Att" tidak ditemukan, artinya semua Tec, Tac, Def, Cre juga tidak ada
            print(f"Tidak ada data ATT untuk: {link}")
            nggak_ada.append(link)

            pemain_data["Att"] = None
            pemain_data["Tec"] = None
            pemain_data["Tac"] = None
            pemain_data["Def"] = None
            pemain_data["Cre"] = None

            # Ambil data alternatif seperti Rating, Total Bermain, dst
            for key, xpath in xpath_dict_alternative.items():
                try:
                    pemain_data[key] = driver.find_element(By.XPATH, xpath).text
                except NoSuchElementException:
                    pemain_data[key] = None

        hasil_data.append(pemain_data)

        print(pemain_data)
    
    nama_klub = file.replace(".csv", "")
    pd.DataFrame(hasil_data).to_csv(f"Data Pemain/{nama_klub}-PEMAIN.csv", index=False)

driver.quit()

Persib Bandung.csv
{'Link Pemain': 'https://www.sofascore.com/id/football/player/dimas-drajad/1567998', 'Att': '58', 'Nama': 'Dimas Drajad', 'Tec': '59', 'Tac': '48', 'Def': '35', 'Cre': '45', 'Total Bermain': '14', 'Starting Eleven': '8', 'Total Menit': '766'}
{'Link Pemain': 'https://www.sofascore.com/id/football/player/ciro-alves/47705', 'Att': '64', 'Nama': 'Ciro Alves', 'Tec': '75', 'Tac': '42', 'Def': '33', 'Cre': '53', 'Total Bermain': '30', 'Starting Eleven': '30', 'Total Menit': '2700'}
Dewa United Fc.csv
{'Link Pemain': 'https://www.sofascore.com/id/football/player/septian-bagaskara/1021948', 'Att': '56', 'Nama': 'Septian Bagaskara', 'Tec': '52', 'Tac': '49', 'Def': '40', 'Cre': '34', 'Total Bermain': '31', 'Starting Eleven': '8', 'Total Menit': '1199'}
{'Link Pemain': 'https://www.sofascore.com/id/football/player/alex/1104491', 'Att': '73', 'Nama': 'Alex', 'Tec': '70', 'Tac': '53', 'Def': '28', 'Cre': '42', 'Total Bermain': '25', 'Starting Eleven': '25', 'Total Menit': '2250